## MAGENTA Preprocessing

### We are going to create the workspace for processing raw data.

We create symbolic links to our data folder in the MAGENTA workspace.

In [2]:
# Create directory paths 
magenta_path = f"$HOME/MAGENTA"
rawdata_path = "/files/magenta/rawdata"
rawdata_links_path = f"$HOME/MAGENTA/data/raw"
scripts_path = "/home/jupyter-andres/MAGENTA/scripts"
metadata_path = "/home/jupyter-andres/MAGENTA/metadata/"
trimmed_data_path = f"$HOME/MAGENTA/trimmed"
assemblies_path = "/files/magenta/assemblies"

### Creating a script to make symbolic links from the raw data to the workspace.

In [16]:
# create script file
with open(scripts_path + '/make_symbolic_links_script.sh', 'w') as f:
    f.write(f'''#!/bin/bash

# paths
rawdata_path="{rawdata_path}"
workspace_path="{workspace_path}"

# exploring directories on the rawdata_path
cd $rawdata_path

# Iterate over each subdirectory in the rawdata_path
for dir in $(ls -d */); do
    # Remove the slash to get the directory name.
    dir_name="${{dir%/}}"
    
# Change to the working directory
cd $workspace_path

    # Crear el directorio si no existe en workspace_path
    mkdir -p "$dir_name"
    
    # Crear enlaces simbólicos desde rawdata_path al directorio en workspace_path
    ln -sf "$rawdata_path/$dir_name/"* "$workspace_path/$dir_name/"
done
''')


In [12]:
# Change the script's execution permissions
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/make_symbolic_links_script.sh

#### Run make_symbolic_links_script.sh in the terminal
! bash /home/jupyter-andres/MAGENTA/scripts/make_symbolic_links_script.sh

### Running FastQC
We will now assess the quality of the reads that we downloaded.

In [47]:

# Crear el archivo de script
with open(scripts_path + '/run_fastqc_script.sh', 'w') as f:
    f.write(f'''#!/bin/bash

# Set paths
magenta_path="{magenta_path}"
workspace_path="{rawdata_links_path}"

# Activate conda environment
echo conda activate /miniconda3/envs/metagenomics 

# Change to the working directory
cd $workspace_path

# Iterate over each subdirectory in the workspace_path
for dir in $(ls -d */); do
    # Remove the trailing slash to get the directory name
    dir_name="${{dir%/}}"
    
    # Create a directory for FastQC output if it doesn't exist
    mkdir -p "$magenta_path/results/00.fastqc/${{dir_name}}_fastqc_untrimmed"
    
    # Run FastQC on all FASTQ files in the current subdirectory of workspace_path
    fastqc "$workspace_path/$dir_name/"*.fastq* -o "$magenta_path/results/00.fastqc/${{dir_name}}_fastqc_untrimmed"
    
done
''')


In [33]:
# Change the script's execution permissions
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/run_fastqc_script.sh

#### Run run_fastqc_script.sh in the terminal
! bash /home/jupyter-andres/MAGENTA/scripts/run_fastqc_script.sh

### Trimming with trimgalore
We will now trimming the reads that we downloaded. This tool filters poor quality reads and trims poor quality bases from the specified samples the quality.

In [11]:
# Crear el archivo de script
with open(scripts_path + '/trim_script.sh', 'w') as f:
    f.write('''#!/bin/bash
# Usage: ./trims_cript.sh

# Change to base directory and create necessary subdirectories
cd $(dirname "$(dirname "$(readlink -f $0)")")
mkdir -p trimmed logs/trim

# Loop through each _1.fastq.gz file in the 'raw' folder and its subdirectories
for f1 in /home/jupyter-andres/MAGENTA/data/raw/*/*_1.fastq.gz; do
  # Get the base name (e.g., SRR17658252 from SRR17658252_1.fastq.gz)
  base_name=$(basename "$f1" "_1.fastq.gz")
  
  # Get the directory of the first file
  dir=$(dirname "$f1")

  # Define the second file (_2.fastq.gz) in the same directory
  f2="${dir}/${base_name}_2.fastq.gz"
  
  # Check if both files exist in the same directory
  if [[ -f "$f1" && -f "$f2" ]]; then
    # Check if the files haven't been trimmed yet
    if [[ ! -f "trimmed/${base_name}_1.fastq.gz" ]]; then
      echo $(date) ${base_name}
      
      # Trim the files
      time trim_galore -o trimmed/ --basename ${base_name} --cores 4 \
        --paired --no_report_file "$f1" "$f2"

      # Rename outputs
      mv -v "trimmed/${base_name}_val_1.fq.gz" "trimmed/${base_name}_1.fastq.gz"
      mv -v "trimmed/${base_name}_val_2.fq.gz" "trimmed/${base_name}_2.fastq.gz"
    fi
  else
    echo "Missing pair for ${base_name}, skipping..."
  fi

done >> logs/trim/trim.log 2>&1

''')


In [69]:
#usage sbatch trimgalore.Slurm <files directory>
#example sbatch trimgalore.Slurm trim/trim_error

In [59]:
# Change the script's execution permissions
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/trim_script.sh

activate the enviroment conda activate /miniconda3/envs/trim_galore

#### rerun fastqc in trimmed fastq

In [16]:
# Crear el archivo de script
with open(scripts_path + '/run_fastqc_trimmed_script.sh', 'w') as f:
    f.write(f'''#!/bin/bash

# Change to base directory and create necessary subdirectories
cd $(dirname "$(dirname "$(readlink -f $0)")")
mkdir -p logs/fastqc/

# Activate conda environment
echo conda activate /miniconda3/envs/metagenomics 

# Change to the working directory
cd $workspace_path

# Iterate over each subdirectory in the workspace_path
for dir in $(ls -d */); do
    # Remove the trailing slash to get the directory name
    dir_name="${{dir%/}}"
    
    # Create a directory for FastQC output if it doesn't exist
    mkdir -p "$magenta_path/results/00.fastqc/${{dir_name}}_fastqc_trimmed"
    
    # Run FastQC on all FASTQ files in the current subdirectory of workspace_path
    fastqc "$trimmed_data_path/"*.fastq* -o "$magenta_path/results/00.fastqc/${{dir_name}}_fastqc_trimmed"
    
done >> $magenta_path/logs/fastqc/fastqc_trim.log 2>&1
''')

In [29]:
# Change the script's execution permissions
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/run_fastqc_trimmed_script.sh

## Corremos el ensamble


In [45]:
with open(scripts_path + '/run_assembly_script.sh', 'w') as f:
    f.write('''#!/bin/bash


# Change to base directory and create necessary subdirectories
cd $(dirname "$(dirname "$(readlink -f $0)")")
mkdir -p logs/assembly/ /files/magenta/assemblies/

# Activate conda environment
echo conda activate /miniconda3/envs/metagenomics 

# Loop through each _1.fastq.gz file in the 'raw' folder and its subdirectories
for f1 in /home/jupyter-andres/MAGENTA/trimmed/*_1.fastq.gz; do
  # Get the base name (e.g., SRR17658252 from SRR17658252_1.fastq.gz)
  base_name=$(basename "$f1" "_1.fastq.gz")

  # Get the directory of the first file
  dir=$(dirname "$f1")

  # Define the second file (_2.fastq.gz) in the same directory
  f2="${dir}/${base_name}_2.fastq.gz"
  
  # Check if both files exist in the same directory
  if [[ -f "$f1" && -f "$f2" ]]; then
    # Create output directory for this assembly
    output_dir="/files/magenta/assemblies/${base_name}_assembled"
    mkdir -p "$output_dir"

    # Assemble the files if not already assembled
    if [[ ! -f "${output_dir}/contigs.fasta" ]]; then
      echo "$(date) - Assembling ${base_name}..."
      
      # Run SPAdes
      spades.py --meta -1 "$f1" -2 "$f2" -o "$output_dir"
    else
      echo "$(date) - Assembly already exists for ${base_name}, skipping..."
    fi
  else
    echo "$(date) - Missing pair for ${base_name}, skipping..."
  fi
done >> "logs/assembly/assembly.log" 2>&1
''')


In [19]:
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/run_assembly_script.sh

####  Con megahit

In [8]:
with open(scripts_path + '/run_megahit_script.sh', 'w') as f:
    f.write('''#!/bin/bash

# Define input and output directories
INPUT_DIR=$(realpath "$1")
OUTPUT_DIR=$(realpath "$2")

# Define contigs storage directory
CONTIGS_DIR="/files/magenta/assemblies/contigs"
mkdir -p "$CONTIGS_DIR"

# Check if input directory exists
if [[ ! -d "$INPUT_DIR" ]]; then
    echo "Error: Input directory '$INPUT_DIR' does not exist."
    exit 1
fi

# Create output directory if it doesn't exist
mkdir -p "$OUTPUT_DIR"

# Get only the input folder name (without absolute path)
INPUT_NAME=$(basename "$1")
LOG_FILE="logs/assembly/megahit_${INPUT_NAME}.log"

# Create log directory if it doesn't exist
mkdir -p "$(dirname "$LOG_FILE")"

# Activate conda environment
echo "Activating MEGAHIT environment..."
echo "conda activate /miniconda3/envs/megahit"

# Loop through each _1.fastq.gz file in the input folder
for f1 in "$INPUT_DIR"/*_1.fastq.gz; do
    # Get the base name (e.g., SRR17658252 from SRR17658252_1.fastq.gz)
    base_name=$(basename "$f1" "_1.fastq.gz")

    # Define the second file (_2.fastq.gz)
    f2="${INPUT_DIR}/${base_name}_2.fastq.gz"

    # Check if both files exist
    if [[ -f "$f1" && -f "$f2" ]]; then
        # Define output directory for this assembly
        sample_output_dir="${OUTPUT_DIR}/${base_name}_assembled"

        # Run assembly if it does not already exist
        if [[ ! -f "${sample_output_dir}/${base_name}.contigs.fa" ]]; then
            echo "$(date) - Assembling ${base_name}..."

            # Run MEGAHIT
            megahit -1 "$f1" -2 "$f2" --min-contig-len 500 -t 5 -m 0.2 \
                --presets meta-large --kmin-1pass --out-prefix "$base_name" -o "$sample_output_dir"

            # Check if assembly was successful
            if [[ -f "${sample_output_dir}/${base_name}.contigs.fa" ]]; then
                echo "$(date) - Assembly completed for ${base_name}. Moving contigs..."
                
                # Move contigs to final directory
                mv "${sample_output_dir}/${base_name}.contigs.fa" "$CONTIGS_DIR/${base_name}.contigs.fa"
                
                # Remove assembly directory
                rm -rf "$sample_output_dir"
                echo "$(date) - Removed temporary assembly folder for ${base_name}."
            else
                echo "$(date) - Assembly failed or contigs.fasta not found for ${base_name}."
            fi
        else
            echo "$(date) - Assembly already exists for ${base_name}, skipping..."
        fi
    else
        echo "$(date) - Missing pair for ${base_name}, skipping..."
    fi
done >> "$LOG_FILE" 2>&1
''')


usamos el --kmin-1pass y el --presets meta-large porque hubo problemas de memoria 

In [46]:
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/run_megahit_script.sh

#### Annotation with prokka 

In [65]:
with open(scripts_path + '/run_prokka_script.sh', 'w') as f:
    f.write('''#!/bin/bash

# Usage: ./prokka.sh [-g] -i input_folder -o output_dir

set -e

genomic=false

# Parse arguments
while [[ "$#" -gt 1 ]]; do
  case $1 in
    -g) genomic=true;;
    -i) input_folder="$2"; shift;;
    -o) output_dir="$2"; shift;;
  esac
  shift
done

# Verify parameters
if [ -z "$input_folder" ]; then >&2 echo "ERROR: Missing input folder"; exit 1; fi
if [ -z "$output_dir" ]; then >&2 echo "ERROR: Missing output directory"; exit 1; fi

# Verify that input folder exists
if [[ ! -d "$input_folder" ]]; then >&2 echo "ERROR: Input folder $input_folder doesn't exist"; exit 1; fi

# Check that Prokka command exists
if ! command -v prokka &> /dev/null; then
  >&2 echo "ERROR: Prokka command not found"; exit 1
fi

# Process each FASTA file in the input folder
for inp in "$input_folder"/*.fasta "$input_folder"/*.fa; do
  # Skip if no matching files are found
  if [[ ! -e "$inp" ]]; then continue; fi
  
  # Get filename without extension
  sample_name=$(basename "$inp" | sed 's/\.[^.]*$//')
  out="$output_dir/$sample_name"
  
  echo $(date +"%D %T:") "Annotation of $sample_name with Prokka started"
  
  # Create output directory
  #mkdir -p "$out"
  
  # Run Prokka
  if [[ $genomic == true ]]; then
    prokka --outdir "$out" --prefix "$sample_name" --cpus 5 "$inp" > /dev/null 2>&1
  else
    prokka --outdir "$out" --prefix "$sample_name" --metagenome --cpus 8 "$inp"
  fi
  
  echo $(date +"%D %T:") "Annotation of $sample_name with Prokka finished"
done
''')

In [63]:
! chmod a+x /home/jupyter-andres/MAGENTA/scripts/run_prokka_script.sh

Rawdata: 141 files (70 pair data, except for SRR6056507)
Trimmed: 126 files (With only one file: ERR4833478, SRR17658255, SRR17658259, SRR17658263, SRR17658264, SRR17658268, SRR17658274, SRR17658283, SRR17658288. SRR17658295. SRR17658297, SRR17658298, SRR17658299, SRR24201529) (With different name: SRR17658252 (1,2), SRR17658263)

In [19]:
# contar Archivos crudos Rawdata
! ls -ltr /files/magenta/rawdata/*/* | wc -l

141


In [14]:
# Archivos crudos Rawdata (Sin par)
! ls /files/magenta/rawdata/*/*.fastq.gz | grep -oP "(SRR|ERR)\d+" |  uniq -u #SRR6056507

SRR6056507


In [38]:
# Archivos trimmed (Sin Par)
! ls /home/jupyter-andres/MAGENTA/trimmed/*/* | grep -oP "(SRR|ERR)\d+" |  uniq -u 

#ERR4833478
#SRR17658255
#SRR17658259
#SRR17658263
#SRR17658264
#SRR17658268
#SRR17658274
#SRR17658283
#SRR17658288
#SRR17658295
#SRR17658297
#SRR17658298
#SRR17658299
#SRR24201529

In [45]:
# Verificar archivos ensamblados en assemblies/contigs. 
! ls /files/magenta/assemblies/contigs/*.contigs.fa | grep -oP "(SRR|ERR)\d+" | sort -u

ERR4833474
ERR4833475
ERR4833476
ERR4833477
ERR4833480
ERR4833481
ERR4833482
SRR12150540
SRR17658256
SRR17658257
SRR17658258
SRR17658260
SRR17658261
SRR17658262
SRR17658265


In [35]:
# contar los ensambles completados
! ls /files/magenta/assemblies/contigs/*.contigs.fa | grep -oP "(SRR|ERR)\d+" | sort -u | wc -l

15


In [66]:
# Comando para identificar cuales archivos trimmed fueron ensamblados para liberar espacio
# funciona solo en la terminal
# Una vez que verifiques que los archivos son correctos agrega -delete en el find para borrarlos
! ls /files/magenta/assemblies/contigs/*.contigs.fa | grep -oP "(SRR|ERR)\d+" | sort -u | while read id; do find /home/jupyter-andres/MAGENTA/trimmed/*/* -type f -name "*$id*"; done

#### Corrupted files
- SRR17658263_1 (discarded*)
  
*The file could not be trimmed